In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [12]:
URL_PAGE = 'https://hh.ru/vacancies/dostavschik?page=1&hhtmFrom=vacancy_search_catalog'

In [20]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result

In [21]:
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

In [22]:
def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [23]:
result = parse_hh(URL_PAGE, headers)

Cтраница №1, ссылка: https://hh.ru/vacancies/dostavschik?page=1&hhtmFrom=vacancy_search_catalog
Cтраница №2, ссылка: https://hh.ru/vacancies/dostavschik?page=2&hhtmFrom=vacancy_search_catalog
Cтраница №3, ссылка: https://hh.ru/vacancies/dostavschik?page=3&hhtmFrom=vacancy_search_catalog
Cтраница №4, ссылка: https://hh.ru/vacancies/dostavschik?page=4&hhtmFrom=vacancy_search_catalog
Cтраница №5, ссылка: https://hh.ru/vacancies/dostavschik?page=5&hhtmFrom=vacancy_search_catalog
Cтраница №6, ссылка: https://hh.ru/vacancies/dostavschik?page=6&hhtmFrom=vacancy_search_catalog
Cтраница №7, ссылка: https://hh.ru/vacancies/dostavschik?page=7&hhtmFrom=vacancy_search_catalog
Cтраница №8, ссылка: https://hh.ru/vacancies/dostavschik?page=8&hhtmFrom=vacancy_search_catalog
Cтраница №9, ссылка: https://hh.ru/vacancies/dostavschik?page=9&hhtmFrom=vacancy_search_catalog
Cтраница №10, ссылка: https://hh.ru/vacancies/dostavschik?page=10&hhtmFrom=vacancy_search_catalog
Парсинг завершен


In [24]:
len(result)

461

In [25]:
result

[{'vacancy_name': 'Водитель-курьер',
  'vacancy_salary': 'до 100\u202f000 руб.',
  'min_salary': None,
  'max_salary': 100000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://hh.ru/vacancy/69812816?query=%D0%B4%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D1%89%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Автокурьер',
  'vacancy_salary': 'до 78\u202f000 руб.',
  'min_salary': None,
  'max_salary': 78000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://hh.ru/vacancy/67602665?query=%D0%B4%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D1%89%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Водитель-курьер СберМаркет',
  'vacancy_salary': '64\u202f000 – 134\u202f000 руб.',
  'min_salary': 64000,
  'max_salary': 134000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://hh.ru/vacancy/68929744?query=%D0%B4%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D1%89%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'КУРЬЕР (Ежедневная оплата)',
  'vacancy_salary': 'до 200\u202f000 ру